<a href="https://colab.research.google.com/github/camgenomicmedicine/gm7/blob/main/GM7_assignment_helper_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GM7 assignment helper notebook

This is a jupyter notebook, similar to what you ran with Victor. However, we can get terminal commands to run from a notebook rather than python. Simply add a ! to the start of the line.

For example; if you run the cell below, which contains `ls`, you will run the command in python. If you run the subsequent command, with `! ls` then this will run the unix terminal command ls, and you will get back the contents of the current directory.

In [ ]:
ls

In [ ]:
! ls

We need to install all the packages we need. On your virtual machines during the practical you had this set up for you beforehand, however, now we have to do this from scratch.

In [ ]:
!apt install samtools bcftools tabix

For your report you mght like to report the version of the tool used

In [ ]:
!samtools --version
!bcftools --version

#Upload your data

Please clock on the file manager to the left and upload the zip file for the assignment. This is called `GM7_assignment_data.zip`


To use the data we need to unzip it:

In [ ]:
! unzip GM7_assignment_data.zip

Now we have all our files available to us, we can check that we can see them

In [ ]:
! ls

#Trio Analysis assignment
##Introduction
In this walkthrough we will use some commands from the practical to perform variant calling and analysis on data from whole-genome sequencing (WGS) of trios (father, mother, and child). Each group will analyze one trio, identify Mendelian inheritance patterns, de novo mutations, and assess variant quality and annotations to infer possible disease-related variants. These are example commands only, you may want to alter these to reflect your thoughts on the data.

Our data contains a subset of data covering:
chr11:5100000-5400000

#Part 1: Exploring BAM Files with samtools mpileup
Let's unzip the chromozome file and call some variants.

In [ ]:
! gunzip chr11.fa.gz

In [ ]:
#In the practical you first ran samtools - like the terminal you will have to enter a q to quit viewing the file. Samtools isn't much use for this task.
!samtools mpileup -f chr11.fa NA19239.bam | less -S
!samtools mpileup -f chr11.fa NA19238.bam | less -S
!samtools mpileup -f chr11.fa NA19240.bam | less -S


#Part 2: Variant Calling for a Trio
##Single-sample variant calling for each member of the trio
Instead of samtools in the practical we used bcftools mpileup and bcftools call:

In [ ]:
!bcftools mpileup -f chr11.fa NA19239.bam | bcftools call -m -Oz -o FATHER.vcf.gz
!bcftools mpileup -f chr11.fa NA19238.bam | bcftools call -m -Oz -o MOTHER.vcf.gz
!bcftools mpileup -f chr11.fa NA19240.bam | bcftools call -m -Oz -o CHILD.vcf.gz


##Multi-sample variant calling
To analyze inheritance patterns it is more useful if we call variants jointly for the trio. Check the order of the trio!

In [ ]:
!bcftools mpileup -Ou -f chr11.fa NA19239.bam NA19238.bam NA19240.bam | bcftools call -mv -Oz -o TRIO.vcf.gz


#Part 3: Identifying Mendelian Violations and De Novo Variants
##Extract de novo variants
Use bcftools to filter potential de novo mutations:

In [ ]:
!bcftools view -i 'GT[2]="0/1" && GT[0]="0/0" && GT[1]="0/0"' TRIO.vcf.gz -Oz -o DE_NOVO.vcf.gz


Q1: List positions and alleles of de novo mutations found in the child.
Q2: What is the functional consequence of a specific de novo variant (use annotation tools in Part 5)?

##Filter for Mendelian consistent variants
Identify variants inherited from both parents:

In [ ]:
!bcftools view -i 'GT[2]="1/1" && GT[0]="0/1" && GT[1]="0/1"' TRIO.vcf.gz -Oz -o INHERITED.vcf.gz


If you want to view the vcf files, you can decompress it as follows:

In [ ]:
!gunzip INHERITED.vcf.gz

Identify variants where the child inherited an allele from a specific parent:

In [ ]:
!bcftools view -i 'GT[0]="1/1" && GT[2]="1/0"' TRIO.vcf.gz -Oz -o PATERNAL_ONLY.vcf.gz
!bcftools view -i 'GT[1]="1/1" && GT[2]="0/1"' TRIO.vcf.gz -Oz -o MATERNAL_ONLY.vcf.gz


#Part 4: Normalization and Annotation of Variants
##Variant normalization
Normalize the VCF for the trio:

In [ ]:
!bcftools norm -f chr11.fa -O z -o TRIO_NORMALIZED.vcf.gz TRIO.vcf.gz


##Annotate variants with rsIDs
Annotate variants using dbSNP - remember, you can use VEP to annotate - you don't really need to do this step.

In [ ]:
#need to get the grch38 database
!wget ftp://ftp.ncbi.nih.gov/snp/organisms/human_9606/VCF/common_all_20180418.vcf.gz

In [ ]:
!tabix -p vcf TRIO_NORMALIZED.vcf.gz


In [ ]:
!tabix -p vcf common_all_20180418.vcf.gz


In [ ]:
!bcftools annotate -c ID -a common_all_20180418.vcf.gz -o TRIO_ANNOTATED.vcf.gz -O z TRIO_NORMALIZED.vcf.gz


Annotate functional impact of variants:

In [ ]:
!bcftools csq -f chr11.fa -g gene_annotation.gff3.gz -Oz -o TRIO_ANNOTATED.vcf.gz TRIO.vcf.gz


#Part 5: Assessing and Filtering Variant Quality
##Calculate statistics
Assess the quality of called variants for the trio:

The bcftools stats commands will not work if you try running them now. This is because the machine your notebook is running on needs some extra programmes to allow it to make plots. You can run them with the following code: (note it will take a couple of minutes to instal)

In [ ]:
!apt install texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-recommended


In [ ]:
!bcftools stats TRIO_NORMALIZED.vcf.gz > TRIO_NORMALIZED.chk
!plot-vcfstats TRIO_NORMALIZED.chk -p trio_stats


##Filter variants
We’ve seen that we can use the TS/TV ratio as a proxy for overall variant quality.
Here we’ll experiment with filtering to improve the overall TS/TV ratio. You can use the
following command to filter the variants, calculate the statistics and retrieve the TS/TV
statistics all in one go.

In [ ]:
!bcftools view -i "QUAL>=50" TRIO_NORMALIZED.vcf.gz | bcftools stats - | grep TSTV | cut -f 5

A very useful command is bcftools filter which allows to "soft filter" the VCF: it can annotate
the FILTER column to indicate sites which fail. Apply the filters discussed above (or use
QUAL>=50 && DP<200) to produce a final call set:

In [ ]:
!bcftools filter -i 'QUAL>=50 && DP<200' -s HighQual TRIO_NORMALIZED.vcf.gz -Oz -o TRIO_FILTERED.vcf.gz


#Part 6: Visualization in IGV
Either download the IGV browser or use the web app!


#Part 7: Investigating Variants of Interest
##Detect Mendelian Inheritance Patterns
Bcftools can directly identify inheritance patterns across trios. You can use the bcftools +mendelian plugin, which flags variants based on Mendelian inheritance rules.

In [ ]:
!bcftools +mendelian TRIO_FILTERED.vcf.gz --trio NA19239,NA19238,NA19240 --output mendelian_violations.txt


We can also add aflag to the INFO field, This will populate a subfield called MERR where 1 indicates violation of Mendelian inheritance.

In [ ]:
!bcftools +mendelian TRIO_FILTERED.vcf.gz --trio NA19239,NA19238,NA19240 -m a -o annotated_trio.vcf